In [1]:
# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# SciPy minimizer routine
from scipy.optimize import minimize
import pickle

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
#

def cost_func(params, ansatz, hamiltonian, estimator, para_root):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    with open(para_root, 'rb') as file:
        para_list = pickle.load(file)
    para_list.append(params)
    with open(para_root, 'wb') as file:
        pickle.dump(para_list, file)
    print(params)
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]

    return cost
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="XXX",
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]

In [2]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import to_ising
from qiskit_optimization.problems import QuadraticProgram
from qiskit.result import QuasiDistribution
from typing import List, Union
from qiskit.quantum_info import Pauli, Statevector
from qiskit.result import QuasiDistribution
import numpy as np
from docplex.mp.model import Model
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import OptimizationResult
from qiskit_optimization.problems.quadratic_program import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization.converters import QuadraticProgramToQubo
import pandas as pd
import random
import sys


# from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.primitives import Sampler
import argparse
import time
import matplotlib.pyplot as plt
import random
import pandas as pd
import os

In [ ]:
def replace_Z_hami(hami, num):
    new = list(hami)
    new[num] = 'Z'
    return ''.join(new)

In [3]:
from typing import List, Union
from docplex.mp.model import Model
from qiskit_optimization.problems.quadratic_program import QuadraticProgram
from qiskit_optimization.applications import OptimizationApplication
from qiskit_algorithms.optimizers import COBYLA
import pandas as pd
import time
from itertools import combinations

class TestCaseOptimization(OptimizationApplication):
    """Optimization application for the "knapsack problem" [1].

    References:
        [1]: "Knapsack problem",
        https://en.wikipedia.org/wiki/Knapsack_problem
    """

    def __init__(self, times: List[float], frs: List[float], w1: float, w2: float, w3: float, sample: List[int],
                 solution: List[int]) -> None:
        """
        Args:
            values: A list of the values of items
            weights: A list of the weights of items
            max_weight: The maximum weight capacity
        """
        self._times = times
        self._frs = frs
        self._w1 = w1
        self._w2 = w2
        self._w3 = w3
        self._sample = sample
        self._solution = solution

    def to_quadratic_program(self) -> QuadraticProgram:
        """Convert a knapsack problem instance into a
        :class:`~qiskit_optimization.problems.QuadraticProgram`

        Returns:
            The :class:`~qiskit_optimization.problems.QuadraticProgram` created
            from the knapsack problem instance.
        """
        mdl = Model(name="TestCaseOptimization")
        x = {i: mdl.binary_var(name=f"x_{i}") for i in self._sample}

        obj_time = 0
        obj_rate = 0
        obj_num = 0

        #         dic_clamp = {}
        for i in range(len(self._solution)):
            if i in self._sample:
                obj_time += self._times[i] * x[i]
                obj_rate += self._frs[i] * x[i]
                obj_num += x[i]
            else:
                obj_time += self._times[i] * self._solution[i]
                obj_rate += self._frs[i] * self._solution[i]
                obj_num += self._solution[i]

        time_sum = sum(self._times)
        rate_sum = sum(self._frs)

        obj_time = pow(obj_time / time_sum, 2)
        obj_rate = pow((obj_rate - rate_sum) / rate_sum, 2)
        obj_num = pow(obj_num / len(self._times), 2)

        obj = self._w1 * obj_time + self._w2 * obj_rate + self._w3 * obj_num

        mdl.minimize(obj)
        op = from_docplex_mp(mdl)
        ising = to_ising(op)

        return op, ising

    def interpret(self, result: Union[OptimizationResult, np.ndarray]) -> List[int]:
        """Interpret a result as item indices

        Args:
            result : The calculated result of the problem

        Returns:
            A list of items whose corresponding variable is 1
        """
        x = self._result_to_x(result)
        return [i for i, value in enumerate(x) if value]

In [4]:
def create_qubo(times, frs, w1, w2, w3, sample, solution):
    testcase = TestCaseOptimization(times, frs, w1, w2, w3, sample, solution)
    op, obj = testcase.to_quadratic_program()
    return op, obj

def get_data(data):
    times = data["time"].values.tolist()
    frs = data["rate"].values.tolist()
    return times, frs

def print_diet(sample,data):
    count = 0
    total_time = 0
    total_rate = 0
    time_list = []
    rate_list = []
    for t in range(len(sample)):
        if sample[t] == 1:
            total_time += data.iloc[t]['time']
            total_rate += data.iloc[t]['rate']
            time_list.append(data.iloc[t]['time'])
            rate_list.append(data.iloc[t]['rate'])
            # print(t[1:]+'. ',end=' ')
            # print('time: '+str(foods[t]['time']), end=', ')
            # print('rate: '+str(foods[t]['rate']), end='\n')
            count += 1
    fval = (1 / 3) * pow(sum(time_list) / sum(data['time']), 2) + (1 / 3) * pow((sum(rate_list) - sum(data["rate"]) + 1e-20) / (sum(data["rate"])+1e-20), 2) + (1 / 3) * pow(count / len(data), 2)
    # print("Total time: " + str(total_time))
    # print("Total rate: " + str(total_rate))
#     print("Fval value:" + str(fval))
#     print("Number: "+str(count))
    return fval

def OrderByImpact(best_solution, df, best_energy):
    impact_values = {}
    for case in range(len(best_solution)):
        if best_solution[case] == 1:
            temp = best_solution.copy()
            temp[case] = 0
            impact_values[case] = 1
            impact_values[case] = print_diet(temp, df) - best_energy
        elif best_solution[case] == 0:
            temp = best_solution.copy()
            temp[case]=1
            impact_values[case] = print_diet(temp, df) - best_energy
    print(impact_values)
    impact_values = sorted(impact_values.items(), key = lambda kv:(kv[1], kv[0]))
    impact_list = []
    for case in impact_values:
        impact_list.append(case[0])
    return impact_list

def OrderByImpactNum(best_solution, df, best_energy):
    num = len(best_solution)
    time_array = list(df["time"])
    rate_array = list(df["rate"])
    time_matrix = np.array(time_array).reshape(-1, 1)
    rate_matrix = np.array(rate_array).reshape(-1, 1)
    num_matrix = np.full((num,1), 1)
    matrix = np.array([best_solution] * len(best_solution))
    for i in range(num):
        if matrix[i][i] == 0:
            matrix[i][i] = 1
        elif matrix[i][i] == 1:
            matrix[i][i] = 0
    time_sum = sum(time_array)
    rate_sum = sum(rate_array)
    # time_sum_con = np.full((len(best_solution), 1), time_sum)
    # rate_sum_con = np.full((len(best_solution), 1), rate_sum)
    time_obj = matrix.dot(time_matrix)
    rate_obj = matrix.dot(rate_matrix) - rate_sum + 1e-20
    num_obj = matrix.dot(num_matrix)
    obj = (1/3)*(time_obj/time_sum)**2 + (1/3)*(rate_obj/(rate_sum+1e-20))**2 + (1/3)*((num_obj)/len(best_solution))**2 - best_energy
    # Get the sorted indices
    sorted_indices = np.argsort(obj, axis=0)

    # Convert the sorted indices to a flattened array
    sorted_indices = sorted_indices.flatten()

    return sorted_indices


def run_alg(qubo, reps):
    global quantum_instance
    # seed = random.randint(1, 9999999)
    # algorithm_globals.random_seed = seed
    optimizer = COBYLA(1)
    qaoa_mes = QAOA(quantum_instance=quantum_instance, optimizer=optimizer, include_custom=True, reps=reps)
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    begin = time.time()
    qaoa_result = qaoa.solve(qubo)
    end = time.time()
    exe_time = end-begin
    return qaoa_result, exe_time

def print_result(result, testcase):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = testcase.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

def plot(fval_list, reps, file_name, problem_size):
    plt.plot(fval_list)
    plt.ylabel('fval')
    plt.savefig("qaoa_"+str(reps)+"/" + file_name + "/size_" + str(problem_size) + "/" + str(num_experiment)+"/fval_trend.png")

def scatter_merge(solution, data):
    time = []
    rate = []
    for t in range(len(solution)):
        if solution[t] == 1.0:
            time.append(data.iloc[t]['time'])
            rate.append(data.iloc[t]['rate'])
    plt.scatter(data["time"], data["rate"], c='red')
    plt.scatter(time, rate)
    plt.show()

def get_initial_fval(length):
    initial_values = [random.choice([0, 1]) for _ in range(length)]
    fval = print_diet(initial_values, df)
    best_solution=initial_values
    best_energy=fval
    return best_solution, best_energy

In [5]:
file_name = "sampled_paintcontrol"
problem_size=7
index_end = problem_size
index_begin = 0
df = pd.DataFrame()
df = pd.read_csv("data/"+file_name+".csv", dtype={"time": float, "rate": float})
length = len(df)
best_solution, best_energy = get_initial_fval(length)
best_itr = 0
impact_order = OrderByImpactNum(best_solution, df, best_energy)
solution = best_solution.copy()
times, frs = get_data(df)
case_list = impact_order[index_begin:index_end]
op, obj = create_qubo(times, frs, 1 / 3, 1 / 3, 1 / 3, case_list, solution)

In [ ]:
from qiskit_aer import AerSimulator

# Step 1. map classical inputs to a quantum problem
# Problem to Hamiltonian operator
# hamiltonian = SparsePauliOp.from_list(hami_list)
hamiltonian = obj[0]
# QAOA ansatz circuit
ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Step 2. Optimize problem for quantum execution
# backend = AerSimulator()
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]
# target = backend.target
# pm = generate_preset_pass_manager(target=target, optimization_level=3)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

ansatz_isa = pm.run(ansatz)

# ISA Obsesrvables
hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
# Step 3: execute using Qiskit Primitives
# session = Session(backend=backend)

# Configure estimator
estimator = Estimator(backend=backend)
estimator.options.default_shots = 1024
# estimator.options.twirling.shots_per_randomization = 1
# estimator.options.twirling.num_randomizations = 1
# estimator.options.twirling.enable_measure = False
estimator.options.optimization_level = 1

# estimator.options.dynamical_decoupling.enable = True

# # Configure sampler
# sampler = Sampler(session=session)
# sampler.options.default_shots = 1024
# sampler.options.dynamical_decoupling.enable = True

x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator), method="COBYLA", options={"maxiter":50})
res

message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.004078745664068058
       x: [ 4.363e+00  8.082e-01]
    nfev: 26
   maxcv: 0.0

In [ ]:
import pickle
# Path to save the pickle file
pickle_file_path = 'result_1.pickle'

try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(res, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
qc_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 1024
result = sampler.run([qc_isa]).result()

# Close the session since we are now done with it
# qc_isa.draw(output="mpl", idle_wires=False, style="iqp")
plot_distribution(result.quasi_dists, figsize=(15, 5))

In [ ]:
samp_dist = result[0].data.meas.get_counts()
samp_dist

In [ ]:
solution = best_solution.copy()
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []
for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution
    print(s+" "+str(samp_dist[s])+" "+str(fval))
print(optimal_fva)
print(optimal_samp)
print(optimal_bits)
print(optimal_solution)

In [ ]:
head_log = ["itr_num", "sub_problem", "fval", "solution", "cobyla_itr", "parameters"]
log_df = pd.DataFrame(columns=head_log)

In [ ]:
itr_num = 1
cobyla_itr = res.nfev
parameters = res.x
values_log = [itr_num, case_list, optimal_fva, optimal_solution, cobyla_itr, parameters]
log_df.loc[len(log_df)] = values_log
log_df.to_csv("qaoa_hardware/"+"log_1.csv")

pickle_file_path = 'qaoa_hardware/primitive_1.pickle'

try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(result, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
head_samp = ["bit_string", "counts", "fval"]
samp_df = pd.DataFrame(columns=head_samp)
solution = best_solution.copy()
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []

for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    value_samp = [s, samp_dist[s], fval]
    samp_df.loc[len(samp_df)] = value_samp
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution
samp_df.to_csv("qaoa_hardware/"+"samp_1.csv")
optimal_fva

In [6]:
# import pandas as pd
#
# # file_name = "sampled_paintcontrol"
# # df = pd.read_csv("data/"+file_name+".csv", dtype={"time": float, "rate": float})
# log1_info = pd.read_csv("qaoa_hardware/log_1.csv")
#
#
# # Step 3: Save the CSV file
#
#
# # case_list = log1_info['sub_problem'][0][1:-1].split(' ')
# # index = [0, 2, 3, 4, 5, 6, 7]
# # case_list = [int(case_list[x]) for x in index
# solution = [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0]
# s = [0,0,1,1,0,0,0]
# case_index = [16,  1, 33, 36, 40, 38, 28]
# for case_index in range(len(case_list)):
#     solution[case_list[case_index]] = int(s[case_index])
# # fval = print_diet(solution, df)
# best_itr1 = [int(x) for x in log1_info['solution'][0][1:-1].split(', ')]
# fval = print_diet(solution, df)
#
# log1_info.at[0, 'solution'] = solution
# log1_info.to_csv("qaoa_hardware/log_1.csv", index=False)
# log1_info
log1_info = pd.read_csv("qaoa_hardware/log_1.csv")
optimal_solution = [int(x) for x in log1_info["solution"][0][1:-1].split(', ')]
optimal_fva = print_diet(optimal_solution, df)

In [7]:
from qiskit_aer import AerSimulator
energy = optimal_fva
solution = optimal_solution
start_impact = time.time()
impact_order = OrderByImpactNum(solution, df, energy)
end_impact = time.time()
impact_time = end_impact-start_impact
index_begin = 0
index_end = problem_size
case_list = impact_order[index_begin:index_end]
op, obj = create_qubo(times, frs, 1 / 3, 1 / 3, 1 / 3, case_list, solution)
# Step 1. map classical inputs to a quantum problem
# Problem to Hamiltonian operator
hamiltonian = obj[0]
# QAOA ansatz circuit
ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Step 2. Optimize problem for quantum execution
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]
# backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_isa = pm.run(ansatz)
# ISA Obsesrvables
hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
# Step 3: execute using Qiskit Primitives

# Configure estimator
estimator = Estimator(backend=backend)
estimator.options.default_shots = 1024
# estimator.options.twirling.shots_per_randomization = 1
# estimator.options.twirling.num_randomizations = 1
# estimator.options.twirling.enable_measure = False
estimator.options.optimization_level = 1

x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

para_root = "qaoa_hardware/para_2.pickle"
para = []
with open(para_root, 'wb') as file:
    pickle.dump(para, file)
res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator, para_root), method="COBYLA", options={"maxiter":50})
pickle_file_path = 'qaoa_hardware/result_2.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(res, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
res

/tmp/ipykernel_941644/3526144959.py:28: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=backend)


[0.90950025 2.89503642]
[1.90950025 2.89503642]
[1.90950025 3.89503642]
[2.80346042 2.44689001]
[2.35648034 2.67096321]
[2.84863092 2.58271467]
[2.18882271 2.19991022]
[2.60038279 2.61608525]
[2.24004424 2.71643448]
[2.12421871 2.76343921]
[2.26277988 2.77465253]
[2.16261127 2.61830633]
[2.24628862 2.77862176]
[2.27037885 2.70892622]
[2.2088206 2.7177176]
[2.25272775 2.70730929]
[2.25437694 2.69177156]
[2.26382056 2.71831339]
[2.24970878 2.70010366]
[2.24961998 2.70967583]
[2.25659314 2.70674577]
[2.25204542 2.70547922]
[2.25365212 2.70762427]
[2.2519438 2.7078916]
[2.25273308 2.70682103]
[2.25258134 2.70750465]
[2.25296736 2.7073561 ]
[2.25269406 2.70721513]
[2.25277483 2.70729244]
[2.25266055 2.70738334]
Dictionary successfully saved to qaoa_hardware/result_2.pickle


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.002369706291902379
       x: [ 2.253e+00  2.707e+00]
    nfev: 30
   maxcv: 0.0

In [8]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
qc_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 1024
result = sampler.run([qc_isa]).result()
pickle_file_path = 'qaoa_hardware/primitive_2.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(result, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
result

/tmp/ipykernel_941644/3764481163.py:9: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)


Dictionary successfully saved to qaoa_hardware/primitive_2.pickle


PrimitiveResult([PubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

In [9]:
samp_dist = result[0].data.meas.get_counts()

In [10]:
head_samp = ["bit_string", "counts", "fval"]
samp_df = pd.DataFrame(columns=head_samp)
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []

for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    value_samp = [s, samp_dist[s], fval]
    samp_df.loc[len(samp_df)] = value_samp
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution.copy()
samp_df.to_csv("qaoa_hardware/"+"samp_2.csv")

itr_num = 2
cobyla_itr = res.nfev
parameters = res.x
head_log = ["itr_num", "sub_problem", "fval", "solution", "cobyla_itr", "parameters", "impact_time"]
log_df = pd.DataFrame(columns=head_log)
values_log = [itr_num, case_list, optimal_fva, optimal_solution, cobyla_itr, parameters, impact_time]
log_df.loc[len(log_df)] = values_log
log_df.to_csv("qaoa_hardware/"+"log_2.csv")

In [11]:
log_info = pd.read_csv("qaoa_hardware/log_2.csv")
optimal_solution = [int(x) for x in log_info["solution"][0][1:-1].split(', ')]
optimal_fva = print_diet(optimal_solution, df)
optimal_fva
# best_itr2 = optimal_solution.copy()

0.1511082650814146

In [12]:
energy = optimal_fva
solution = optimal_solution
start_impact = time.time()
impact_order = OrderByImpactNum(solution, df, energy)
end_impact = time.time()
impact_time = end_impact-start_impact
index_begin = 0
index_end = problem_size
case_list = impact_order[index_begin:index_end]
op, obj = create_qubo(times, frs, 1 / 3, 1 / 3, 1 / 3, case_list, solution)
# Step 1. map classical inputs to a quantum problem
# Problem to Hamiltonian operator
hamiltonian = obj[0]
# QAOA ansatz circuit
ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Step 2. Optimize problem for quantum execution
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]
# backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_isa = pm.run(ansatz)
# ISA Obsesrvables
hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
# Step 3: execute using Qiskit Primitives

# Configure estimator
estimator = Estimator(backend=backend)
estimator.options.default_shots = 1024
# estimator.options.twirling.shots_per_randomization = 1
# estimator.options.twirling.num_randomizations = 1
# estimator.options.twirling.enable_measure = False
estimator.options.optimization_level = 1

x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

para_root = "qaoa_hardware/para_3.pickle"
para = []
with open(para_root, 'wb') as file:
    pickle.dump(para, file)
res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator, para_root), method="COBYLA", options={"maxiter":50})
pickle_file_path = 'qaoa_hardware/result_3.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(res, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
res

/tmp/ipykernel_941644/3998875362.py:27: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=backend)


[5.46927354 2.87198587]
[6.46927354 2.87198587]
[5.46927354 3.87198587]
[4.75993406 2.16711886]
[3.79066986 1.92109667]
[5.01442293 1.73672902]
[4.87025904 2.39145878]
[4.9218271  2.63608244]
[4.75808908 2.44662127]
[5.10405775 2.3029254 ]
[4.78162565 2.47960141]
[4.92846104 2.3686821 ]
[4.83397724 2.34056791]
[4.85186578 2.41672241]
[4.88549646 2.38800023]
[4.90090682 2.38541929]
[4.88420599 2.38029505]
[4.87912429 2.40226684]
[4.88139843 2.38134882]


/home/complexse/anaconda3/envs/ibm_runtime/lib/python3.10/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:862: UserWarning: The backend ibm_brisbane currently has a status of internal.
  warnings.warn(


[4.8892954  2.38709094]
[4.88261687 2.3906397 ]
[4.88457376 2.3862788 ]
[4.88545334 2.38897584]
[4.88641552 2.38767009]
[4.88518544 2.38762381]
[4.88573315 2.38794039]
[4.88540854 2.38822799]
[4.88541932 2.3879366 ]
[4.88552828 2.38796166]
[4.88546673 2.38809571]
Dictionary successfully saved to qaoa_hardware/result_3.pickle


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.000714157416174381
       x: [ 4.885e+00  2.388e+00]
    nfev: 30
   maxcv: 0.0

In [13]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
qc_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 1024
result = sampler.run([qc_isa]).result()
pickle_file_path = 'qaoa_hardware/primitive_3.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(result, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
result

/tmp/ipykernel_941644/824132600.py:8: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)


Dictionary successfully saved to qaoa_hardware/primitive_3.pickle


PrimitiveResult([PubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

In [14]:
samp_dist = result[0].data.meas.get_counts()

In [15]:
samp_df = pd.DataFrame(columns=head_samp)
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []

for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    value_samp = [s, samp_dist[s], fval]
    samp_df.loc[len(samp_df)] = value_samp
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution.copy()
samp_df.to_csv("qaoa_hardware/"+"samp_3.csv")

itr_num = 3
cobyla_itr = res.nfev
parameters = res.x
head_log = ["itr_num", "sub_problem", "fval", "solution", "cobyla_itr", "parameters", "impact_time"]
log_df = pd.DataFrame(columns=head_log)
values_log = [itr_num, case_list, optimal_fva, optimal_solution, cobyla_itr, parameters, impact_time]
log_df.loc[len(log_df)] = values_log
log_df.to_csv("qaoa_hardware/"+"log_3.csv")

In [16]:
log_info = pd.read_csv("qaoa_hardware/log_3.csv")
optimal_solution = [int(x) for x in log_info["solution"][0][1:-1].split(', ')]
optimal_fva = print_diet(optimal_solution, df)
optimal_fva

0.13262623255985026

In [17]:
energy = optimal_fva
solution = optimal_solution
start_impact = time.time()
impact_order = OrderByImpactNum(solution, df, energy)
end_impact = time.time()
impact_time = end_impact-start_impact
index_begin = 0
index_end = problem_size
case_list = impact_order[index_begin:index_end]
op, obj = create_qubo(times, frs, 1 / 3, 1 / 3, 1 / 3, case_list, solution)
# Step 1. map classical inputs to a quantum problem
# Problem to Hamiltonian operator
hamiltonian = obj[0]
# QAOA ansatz circuit
ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Step 2. Optimize problem for quantum execution
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]
# backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_isa = pm.run(ansatz)
# ISA Obsesrvables
hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
# Step 3: execute using Qiskit Primitives

# Configure estimator
estimator = Estimator(backend=backend)
estimator.options.default_shots = 1024
# estimator.options.twirling.shots_per_randomization = 1
# estimator.options.twirling.num_randomizations = 1
# estimator.options.twirling.enable_measure = False
estimator.options.optimization_level = 1

x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

para_root = "qaoa_hardware/para_4.pickle"
para = []
with open(para_root, 'wb') as file:
    pickle.dump(para, file)
res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator, para_root), method="COBYLA", options={"maxiter":50})
pickle_file_path = 'qaoa_hardware/result_4.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(res, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
res

/tmp/ipykernel_941644/2357403478.py:27: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=backend)


[3.06163448 1.96855204]
[4.06163448 1.96855204]
[3.06163448 2.96855204]
[2.0620125  1.94105859]
[2.56182349 1.95480531]
[2.0632839  1.91661791]
[2.56869685 1.70489982]
[2.06758788 2.03050977]
[2.31470569 1.99265754]
[2.68384399 1.98193461]
[2.56140987 1.89230668]
[2.55277718 1.83040574]
[2.53016056 1.89251349]
[2.61863714 1.86718121]
[2.53384739 1.8775801 ]
[2.56164733 1.90792988]
[2.5714857  1.88036439]
[2.55419277 1.88931527]
[2.56500047 1.89076837]
[2.56160316 1.89620815]
[2.55984664 1.89113578]
[2.56167209 1.89324738]
[2.56215529 1.89167579]
[2.56099872 1.8920433 ]
[2.5615386  1.89251413]
[2.56157763 1.8921293 ]
[2.56131016 1.89231431]
[2.56140606 1.89225683]
[2.56145021 1.89239819]
Dictionary successfully saved to qaoa_hardware/result_4.pickle


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.0008952603081070199
       x: [ 2.561e+00  1.892e+00]
    nfev: 29
   maxcv: 0.0

In [18]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
qc_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 1024
result = sampler.run([qc_isa]).result()
pickle_file_path = 'qaoa_hardware/primitive_4.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(result, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
result

/tmp/ipykernel_941644/3630223236.py:8: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)


Dictionary successfully saved to qaoa_hardware/primitive_4.pickle


PrimitiveResult([PubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

In [19]:
samp_dist = result[0].data.meas.get_counts()

In [20]:
samp_df = pd.DataFrame(columns=head_samp)
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []

for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    value_samp = [s, samp_dist[s], fval]
    samp_df.loc[len(samp_df)] = value_samp
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution.copy()
samp_df.to_csv("qaoa_hardware/"+"samp_4.csv")

itr_num = 4
cobyla_itr = res.nfev
parameters = res.x
head_log = ["itr_num", "sub_problem", "fval", "solution", "cobyla_itr", "parameters", "impact_time"]
log_df = pd.DataFrame(columns=head_log)
values_log = [itr_num, case_list, optimal_fva, optimal_solution, cobyla_itr, parameters, impact_time]
log_df.loc[len(log_df)] = values_log
log_df.to_csv("qaoa_hardware/"+"log_4.csv")

In [21]:
log_info = pd.read_csv("qaoa_hardware/log_4.csv")
optimal_solution = [int(x) for x in log_info["solution"][0][1:-1].split(', ')]
optimal_fva = print_diet(optimal_solution, df)
optimal_fva

0.12876449798027484

In [22]:
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="db8676fa00b1c67b3b2c1f4d317be3ff6c36a156e751c66f7609f3e5cb34e18eb33f5619d82489ef3702d8122af5f58c5bffb9856baab9132ea4bc6f96028bf7",
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]

energy = optimal_fva
solution = optimal_solution
start_impact = time.time()
impact_order = OrderByImpactNum(solution, df, energy)
end_impact = time.time()
impact_time = end_impact-start_impact
index_begin = 0
index_end = problem_size
case_list = impact_order[index_begin:index_end]
op, obj = create_qubo(times, frs, 1 / 3, 1 / 3, 1 / 3, case_list, solution)
# Step 1. map classical inputs to a quantum problem
# Problem to Hamiltonian operator
hamiltonian = obj[0]
# QAOA ansatz circuit
ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Step 2. Optimize problem for quantum execution
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backends(name="ibm_brisbane")[0]
# backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
ansatz_isa = pm.run(ansatz)
# ISA Obsesrvables
hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)
# Step 3: execute using Qiskit Primitives

# Configure estimator
estimator = Estimator(backend=backend)
estimator.options.default_shots = 1024
# estimator.options.twirling.shots_per_randomization = 1
# estimator.options.twirling.num_randomizations = 1
# estimator.options.twirling.enable_measure = False
estimator.options.optimization_level = 1

x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

para_root = "qaoa_hardware/para_5.pickle"
para = []
with open(para_root, 'wb') as file:
    pickle.dump(para, file)
res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator, para_root), method="COBYLA", options={"maxiter":50})
pickle_file_path = 'qaoa_hardware/result_5.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(res, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
res

/tmp/ipykernel_941644/2442508433.py:38: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=backend)


[5.81067473 5.35922983]
[6.81067473 5.35922983]
[6.81067473 6.35922983]
[7.71764197 4.93802866]
[7.26415835 5.14862924]
[6.56512682 5.31225936]
[6.85959415 5.60439689]
[6.92482382 5.30828884]
[6.81412627 5.42163445]
[6.77054007 5.31131888]
[6.83976835 5.34782264]
[6.79563025 5.35501033]
[6.81198361 5.37479991]
[6.81837884 5.35793295]
[6.81015147 5.36310087]
[6.80799231 5.35639021]
[6.81260986 5.35896532]
[6.80974531 5.35893008]
[6.81063579 5.36020561]
[6.81097248 5.35884284]
[6.8106504  5.35947275]
[6.81043483 5.35918449]
[6.81077473 5.35922888]
[6.81067426 5.35917983]
[6.81064483 5.35932525]
Dictionary successfully saved to qaoa_hardware/result_5.pickle


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 6.085743553679383e-05
       x: [ 6.811e+00  5.359e+00]
    nfev: 25
   maxcv: 0.0

In [23]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
qc_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 1024
result = sampler.run([qc_isa]).result()
pickle_file_path = 'qaoa_hardware/primitive_5.pickle'
try:
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(result, file)
    print(f"Dictionary successfully saved to {pickle_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
result

/tmp/ipykernel_941644/1660856653.py:8: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)


Dictionary successfully saved to qaoa_hardware/primitive_5.pickle


PrimitiveResult([PubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

In [24]:
samp_dist = result[0].data.meas.get_counts()

In [25]:
samp_df = pd.DataFrame(columns=head_samp)
optimal_fva = print_diet(best_solution, df)
optimal_bits = ''
optimal_samp = 0
optimal_solution = []

for s in samp_dist.keys():
    for case_index in range(len(case_list)):
        solution[case_list[case_index]] = int(s[case_index])
    fval = print_diet(solution, df)
    value_samp = [s, samp_dist[s], fval]
    samp_df.loc[len(samp_df)] = value_samp
    if fval < optimal_fva:
        optimal_fva = fval
        optimal_bits = s
        optimal_samp = samp_dist[s]
        optimal_solution = solution.copy()
samp_df.to_csv("qaoa_hardware/"+"samp_5.csv")

itr_num = 5
cobyla_itr = res.nfev
parameters = res.x
head_log = ["itr_num", "sub_problem", "fval", "solution", "cobyla_itr", "parameters", "impact_time"]
log_df = pd.DataFrame(columns=head_log)
values_log = [itr_num, case_list, optimal_fva, optimal_solution, cobyla_itr, parameters, impact_time]
log_df.loc[len(log_df)] = values_log
log_df.to_csv("qaoa_hardware/"+"log_5.csv")

In [26]:
log_info = pd.read_csv("qaoa_hardware/log_5.csv")
optimal_solution = [int(x) for x in log_info["solution"][0][1:-1].split(', ')]
optimal_fva = print_diet(optimal_solution, df)
optimal_fva

0.12748066282974857